# Crimes and Samples graph

### This code reads a csv file, defines independent and dependent variables, runs 300 iterations of Machine Learning models with different train-test splits and displays a histogram and mean of the Mean Squared Error results  

In [7]:
import pandas as pd
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import json
import os


from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
		accuracy_score as acc_score,
		recall_score   as recall,
		confusion_matrix,
        mean_squared_error, 
        r2_score
		)
from sklearn.decomposition import PCA


#MODELS
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
#import xgboost as xgb
#xgb_regressor = xgb.XGBRegressor()
from sklearn.neural_network import MLPRegressor  # Use MLPRegressor for regression
import xgboost as xgb

#from minepy import MINE

In [8]:
#Path
#Insert your local repo path to file
path_home = "D:\Archivos\Social Data Challenge\data\grids\quarter_samples"

csv_100 = pd.read_csv(path_home + '\grid_100_htmp.csv')
csv_500 = pd.read_csv(path_home + '\grid_500_htmp.csv')

In [9]:
csv_100.head()

,id,lat,lon,metro_100,cams_100,zonas_pat,antro_100,salud_100,museo_100,esc_100,...,crime22q1,sample22q1,crime22q2,sample22q2,crime22q3,sample22q3,crime22q4,sample22q4,crime23q1,sample23q1
0,1,19.465514,-99.184187,0,0,1,0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,2,19.464666,-99.184187,0,0,1,0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
2,3,19.463818,-99.184187,0,0,1,0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
3,4,19.462970,-99.184187,0,0,1,0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
4,5,19.462123,-99.184187,0,0,1,0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [10]:
#x =  csv_100.iloc[:, 1:-2]

df = csv_100.copy()


y_crime23q1 = csv_100['crime23q1']
y_sample23q1 = csv_100['sample23q1']

#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)
#x_1=pd.DataFrame(X_scaled, columns = X.columns)

#X_scaled[X_scaled.sample22q2 != 0]
#x[x.crime22q4 >0]

### Select if data will be scaled and if Maximal Information Coefficient and PCA Analysis will be used to adjust X. Also, create the two dependent variables

In [11]:
scaled= False
mic = False
pca_flag = False
pca_components = 1

#x =  df.iloc[:, 1:-2]
x =  df.iloc[:, 3:-2]

if scaled :
    scaler = StandardScaler()
    x =  pd.DataFrame(scaler.fit_transform(x), columns = x.columns)


if mic:
    mic_cols= [col for col in x.columns if col.startswith('sample')]
    x= x[mic_cols]

if pca_flag:
    pca = PCA(n_components=pca_components)
    pca.fit(x)
    x=pd.DataFrame(pca.transform(x))
    
#x_to_use= x_scaled

y_crimes = df['crime23q1']
y_samples = df['sample23q1']
#y =  y_crime23q1




In [13]:
scaler = StandardScaler()
x_scaled =  pd.DataFrame(scaler.fit_transform(x), columns = x.columns)


In [14]:
models = {
    'Linear Regression': LinearRegression(),
    #'RF Crimes Scaled': RandomForestRegressor(n_estimators =200,max_depth= 20),
    #'RF Samples Scaled': RandomForestRegressor(n_estimators =200,max_depth= 20)#,
    # 'SVR Crimes Scaled': SVR(C=10, kernel='rbf'),
    # 'SVR Samples Scaled': SVR(C=10, kernel='rbf')#,
    #'XGBoost Crimes Scaled':  xgb.XGBRegressor(),
    #'XGBoost Samples Scaled': xgb.XGBRegressor()

}

# Create a DataFrame to store the results
results = {'Model': [], 'Average MSE': []}
# Create a dictionary to store MSE values for each model
mse_values_dict = {model_name: [] for model_name in models.keys()}


# Perform iterations for each model
num_iterations = 300




In [15]:
for model_name, model in models.items():
    
    mse_values = []
    x = x_scaled   if 'Scaled'  in model_name else x
    y = y_samples  if 'Samples' in model_name else y_crimes


    for _ in range(num_iterations):
         
            
        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=None)
        
        # Fit the model
        model.fit(X_train, y_train)
        
        # Make predictions on the test set
        y_pred = model.predict(X_test)
        
        # Calculate MSE for this iteration
        mse = mean_squared_error(y_test, y_pred)
        mse_values.append(mse)
        
        # Store the MSE values for this model in the dictionary
        mse_values_dict[model_name] = mse_values

    # Calculate the average MSE
    average_mse = np.mean(mse_values)
    
    # Append results to the DataFrame
    results['Model'].append(model_name)
    results['Average MSE'].append(average_mse)
    
# Create a DataFrame from the results
results_df = pd.DataFrame(results)


In [ ]:
# Plot histograms for each model's MSE
plt.figure(figsize=(12, 6))
for model_name in models.keys():
    plt.hist(mse_values_dict[model_name], bins=20, alpha=0.6, label=model_name)

plt.xlabel('Mean Squared Error (MSE)')
plt.ylabel('Frequency')
plt.title('Distribution of MSE for Each Model')
plt.legend()
plt.grid(True)

# Show the histogram plot
plt.show()

# Display the results DataFrame
print(results_df)

In [ ]:
# Initialize a dictionary to store the best models and their scores
best_models = {}
best_scores = {}
best_parameters = {}

# Create a list to store the results
results = []

# Create lists to store MSE values for each model
mse_values = {model_name: [] for model_name in models}


# Iterate through the models and perform grid search
for model_name, model in models.items():
    
    average_mse = 0.0
    
    for _ in range(300):  # 300 iterations
        X_train, X_test, y_train, y_test = 
            train_test_split(
                x, y, test_size=0.2, random_state=None)
        

        param_grid = param_grids[model_name]
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)

        # Get the best model and its score
        best_model = grid_search.best_estimator_
        #best_score = -grid_search.best_score_  # Negative MSE
        best_params = grid_search.best_params_

        y_pred = best_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)

        # Store the MSE value
        mse_values[model_name].append(mse)
        
    # Calculate the average MSE for this model
    average_mse = np.mean(mse_values[model_name])
    
     # Append results to the list
    results.append({
        'Model': model_name,
        'Best_Parameters': best_params,
        'Best_MSE': -grid_search.best_score_,  # GridSearchCV returns negative MSE
        'Average_MSE': average_mse
    })
    
# Create a DataFrame to store the results
results_df = pd.DataFrame(results)


 


results_df.sort_values(by='Best MSE Score', ascending=True, inplace=True)
print(results_df)

print(best_models)
print(best_scores)
print(best_parameters)


In [ ]:
results_df